### Dataset & DataLoader 살펴보기
- Pytorch에서 배치크기만큼 데이터를 조절하기 위한 메커니즘
- Dataset : 사용 데이터를 기반으로 사용자정의 클래스 작성
- DataLoader : 지정된 Dataset에서 지정된 batch size만큼 피쳐와 타겟을 추출하여 전달

[1] 모듈 로딩 및 데이터 준비 <hr>

In [180]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import numpy as np
import pandas as pd

[2] 데이터셋 생성<hr>
- 사용자 정의 데이터셋 생성

In [181]:
filename = '../data/iris.csv'

# pandas
irisDF = pd.read_csv(filename)
irisDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal.length  150 non-null    float64
 1   sepal.width   150 non-null    float64
 2   petal.length  150 non-null    float64
 3   petal.width   150 non-null    float64
 4   variety       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [204]:
# 사용자 정의 Dataset 클래스
# - 데이터의 Tensor 변환
class DLDataset(Dataset):
    
    # 초기화 함수
    def __init__(self,x_data,y_data):
        super().__init__()
        # x, y data => ndarray
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data
        
        # ndarray => tensor : 텐서화
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data)
        
    # 데이터셋의 갯수 체크 함수
    def __len__(self):
        return self.target.shape[0]
        pass
    
    # 특정 인덱스 데이터 + 라벨 반환 콜백 함수 - 튜플로 반환
    def __getitem__(self,index):
        return self.feature[index],self.target[index]
        pass

In [183]:
# 피쳐와 라벨로 분리
featureDF = irisDF[irisDF.columns[:-1]]
targetDF = irisDF[irisDF.columns[-1]]  
# target이 문자열이라 인코딩 필요 -> replace or encoder 이용
# 2차원이어야 하니까 .to_frame() 해야함 / 밑에 라벨인코더 사용하기 위해 1차원으로 놔둠 / 라벨인코더는 1차원으로 받음

In [184]:
from sklearn.preprocessing import LabelEncoder

# object 타입 타겟 => int 타입 타겟
targetNP = LabelEncoder().fit_transform(targetDF)

# 차원늘리기
targetNP = targetNP.reshape(-1,1)
targetNP.shape, targetNP.ndim

((150, 1), 2)

In [185]:
# 데이터셋 생성 -> DF, NP
my_dataset = DLDataset(featureDF,targetNP)

In [186]:
my_dataset[0], featureDF.iloc[0], targetDF[0]

((tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0])),
 sepal.length    5.1
 sepal.width     3.5
 petal.length    1.4
 petal.width     0.2
 Name: 0, dtype: float64,
 'Setosa')

- [2-3] 학습용, 검증용, 테스트용 Dataset<hr>

In [187]:
# 파이토치
from torch.utils.data import random_split

# 학습용, 검증용, 테스트용 데이터 비율
seed = torch.Generator().manual_seed(42)
trainDS, validDS, testDS = random_split(my_dataset, [0.7,0.1,0.2], generator=seed) # generator : 랜덤 씨드 설정

print(f"train_ds: {len(trainDS)}개, val_ds: {len(validDS)}개, test_ds: {len(testDS)}개")
# Subset 타입 
# Subset 속성 - indices, dataset
trainDS.indices, validDS.indices, testDS.indices  # 빼온 데이터의 인덱스

train_ds: 105개, val_ds: 15개, test_ds: 30개


([42,
  95,
  30,
  64,
  52,
  35,
  130,
  40,
  82,
  17,
  108,
  94,
  68,
  97,
  117,
  127,
  41,
  44,
  57,
  140,
  149,
  32,
  23,
  102,
  16,
  113,
  71,
  18,
  67,
  66,
  0,
  25,
  101,
  112,
  91,
  3,
  59,
  116,
  86,
  84,
  106,
  142,
  43,
  39,
  26,
  98,
  93,
  20,
  87,
  19,
  120,
  114,
  7,
  63,
  76,
  89,
  36,
  45,
  37,
  56,
  58,
  122,
  51,
  145,
  24,
  21,
  105,
  62,
  15,
  11,
  48,
  133,
  88,
  50,
  6,
  134,
  111,
  8,
  49,
  75,
  69,
  124,
  4,
  147,
  80,
  100,
  99,
  141,
  47,
  107,
  13,
  109,
  129,
  28,
  38,
  53,
  121,
  5,
  55,
  31,
  73,
  74,
  54,
  29,
  12],
 [22, 104, 81, 1, 103, 125, 85, 2, 96, 128, 27, 118, 77, 110, 146],
 [72,
  139,
  131,
  60,
  65,
  92,
  135,
  83,
  14,
  34,
  137,
  10,
  119,
  9,
  148,
  79,
  78,
  70,
  144,
  143,
  123,
  115,
  61,
  132,
  90,
  46,
  126,
  136,
  33,
  138])

[3] DataLoder 생성<hr>

In [188]:
# DataLoder 생성
# drop_last 매개변수 : 배치사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 [기본 : False]
batch = 10
trainDL = DataLoader(trainDS, batch_size=batch)
validDL = DataLoader(validDS, batch_size=batch)
testDL = DataLoader(testDS, batch_size=batch)

In [189]:
# Epoch당 데이터 뽑아내는 횟수 
# 5개(batch size)씩 21번 / 3번 / 6번
# if batch size = 10 -> 10번 / 2번 / 3번
len(trainDL), len(validDL), len(testDL) 

(11, 2, 3)

In [190]:
# DataLoder 속성
for _,(feature,target) in enumerate(validDL):
    print(f'[{_}] feature: {feature}, target: {target}')
    # 로더에서 가지고 온 데이터만큼 학습 진행

[0] feature: tensor([[4.6000, 3.6000, 1.0000, 0.2000],
        [6.5000, 3.0000, 5.8000, 2.2000],
        [5.5000, 2.4000, 3.7000, 1.0000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [6.3000, 2.9000, 5.6000, 1.8000],
        [7.2000, 3.2000, 6.0000, 1.8000],
        [6.0000, 3.4000, 4.5000, 1.6000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [5.7000, 2.9000, 4.2000, 1.3000],
        [6.4000, 2.8000, 5.6000, 2.1000]]), target: tensor([[0],
        [2],
        [1],
        [0],
        [2],
        [2],
        [1],
        [0],
        [1],
        [2]])
[1] feature: tensor([[5.2000, 3.5000, 1.5000, 0.2000],
        [7.7000, 2.6000, 6.9000, 2.3000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [6.5000, 3.2000, 5.1000, 2.0000],
        [6.3000, 2.5000, 5.0000, 1.9000]]), target: tensor([[0],
        [2],
        [1],
        [2],
        [2]])


[4] Model 클래스 정의<hr>
- 입/출력 피쳐 수, 층 수, 은닉층의 노드 수
- 구조 설계
    * 입력층 : 입력 = 피쳐 갯수 -> iris 4개
    * 은닉층 : 마음대로 알아서 잘
    * 출력층 : 출력 = [분류] 타겟 클래스 갯수 -> iris 3개 [회귀] 1개

In [191]:
import torch.nn.functional as F

In [192]:
import torch.nn as nn

class Iris(nn.Module):
    
    # 모델 구조 설정
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(4, 10)
        self.layer2 = nn.ReLU()  # 입력 차원을 지정하지 않음
        self.layer3 = nn.Linear(10, 3)  # 입력 차원을 지정
        self.softmax = nn.Softmax(dim=1)  # 클래스 수에 해당하는 출력 차원 지정
        
    # 순방향 학습 진행 콜백 함수
    def forward(self, x): # 입력데이터 : x
        y = self.layer1(x)
        y = self.layer2(y)
        y = self.layer3(y)
        y = self.softmax(y)
        return y


In [206]:
# 모델 클래스 정의
# 클래스명 : CModel
class CModel(nn.Module):
    
    # 모델 구조 설정
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.input_layer = nn.Linear(in_dim, 100)
        self.relu = nn.ReLU()  # 0보다 작거나 같으면 0 / 0보다 크면 1
        self.hidden_layer = nn.Linear(100, 27) 
        self.output_layer = nn.Linear(27, out_dim) 
        
    # 순방향 학습 진행 콜백 함수
    def forward(self, x): # 입력데이터 : x
        y = self.input_layer(x)  
        y = self.relu(y)      
        y = self.hidden_layer(y) 
        y = self.relu(y)        
        y = self.output_layer(y)  
        
        return y


[5] 학습 준비 <hr>
- 실행 디바이스, 모델, 최적화, 손실함수, 학습횟수, 학습함수, 평가함수, 예측함수

In [194]:
# 실행 디바이스 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 학습 횟수
EPOCHS = 50

In [195]:
# 모델 인스턴스
IN_DIM = my_dataset.feature.shape[1]
OUT_DIM = len(torch.unique(my_dataset.target))  # or len(np.unique(targetNP) or tragetDF.nunique()

model = CModel(IN_DIM, OUT_DIM).to(DEVICE)

In [196]:
# 손실함수
LOSS_FUNCTION = nn.CrossEntropyLoss().to(DEVICE)

# 최적화 인스턴스
import torch.optim as optim
OPTIMIZER = optim.Adam(model.parameters())

# 스케쥴러
from torch.optim.lr_scheduler import ReduceLROnPlateau
SCHEDULER = ReduceLROnPlateau(OPTIMIZER, 'min', patience=10)

- 학습 및 검증관련 함수 정의

In [197]:
import torchmetrics.functional as metrics  

In [198]:
# 학습 진행함수
def training(dataLoader):
    # 학습모드 => 정규화, 경사하강법, 드랍아웃 등의 기능 활성화
    model.train() 
    
    # 배치크기만큼 학습진행 및 저장
    train_loss = []
    train_acc = []
    for cnt, (feature, target) in enumerate(dataLoader):
        feature, target = feature.to(DEVICE), target.to(DEVICE)
        target = target.squeeze()
        
        # 학습
        pre_target = model(feature)

        # 손실계산
        loss = LOSS_FUNCTION(pre_target, target)
        train_loss.append(loss)
        
        # 정확도 
        acc = metrics.accuracy(pre_target, target, task = 'multiclass', num_classes=3)
        train_acc.append(acc)
        
        # W, b 업데이트 
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()
        
        # 배치 사이즈 단위 학습 진행 메시지 출력
        # print(f'[{cnt}] : [Train batch loss] ==> {loss}')

    # 에포크 단위 학습 진행 메시지 출력
    print(f'[Train loss] ==> {loss}    [Train Accuracy] ==> {acc}')
    
    return train_loss, train_acc

In [199]:
train_loss, train_acc = training(trainDL)

[Train loss] ==> 1.0590155124664307    [Train Accuracy] ==> 0.6000000238418579


In [200]:
# 검증 및 평가 진행 함수
# 매개변수 dataLoader : 검증 또는 테승트 데이터셋에 대한 Loader
def testing(dataLoader):
    # 학습모드 => 정규화, 경사하강법, 드랍아웃 등의 기능 비활성화
    model.eval()
    
    with torch.no_grad():
        # 배치크기만큼 학습진행
        val_loss = []
        val_acc = []
        for cnt, (feature, target) in enumerate(dataLoader): 
            feature, target = feature.to(DEVICE), target.to(DEVICE)
            target = target.squeeze()
            
            # 학습
            pre_target = model(feature)
    
            # 손실계산
            loss = LOSS_FUNCTION(pre_target, target)
            val_loss.append(loss)
            
            # 정확도 
            acc = metrics.accuracy(pre_target, target, task = 'multiclass', num_classes=3)
            val_acc.append(acc)
            
            # 배치 사이즈 단위 학습 진행 메시지 출력
            # print(f'[{cnt}] : [Train batch loss] ==> {loss}')

    # 에포크 단위 학습 진행 메시지 출력
    print(f'[Valid loss] ==> {loss}    [Valid Accuracy] ==> {acc}')
    
    return val_loss, val_acc

In [201]:
# 예측 함수
def predict():
    pass

[6] 학습 진행 <hr>

In [203]:
# 지정된 횟수만큼 처음부터 끝까지 학습 및 검증 진행
# 목표 : 최적(Error 최소화)의 W, b를 가진 모델 완성
# 
for eps in range(EPOCHS):
    # 학습
    train_loss, train_acc = training(trainDL)
    
    # 검증
    val_loss, val_acc = testing(validDL)
    
    print(f'[{eps+1}/{EPOCHS}]\nTRAIN : {sum(train_loss)/len(train_loss)}\nVAL : {sum(val_loss)/len(val_loss)}')
    
    # 조기 종료 기능 => 조건 : val_loss가 지정된 횟수 이상 개선이 안되면 학습 종료
    if SCHEDULER.num_bad_epochs > SCHEDULER.patience:
        print(f"Early stopping at epoch {eps}")
        break

[Train loss] ==> 0.8949167132377625    [Train Accuracy] ==> 1.0
[Valid loss] ==> 0.9000127911567688    [Valid Accuracy] ==> 0.6000000238418579
[1/50]
TRAIN : 0.9388335943222046
VAL : 0.8939346075057983
[Train loss] ==> 0.8172163963317871    [Train Accuracy] ==> 1.0
[Valid loss] ==> 0.8043423891067505    [Valid Accuracy] ==> 0.6000000238418579
[2/50]
TRAIN : 0.8497505187988281
VAL : 0.7992217540740967
[Train loss] ==> 0.7301682233810425    [Train Accuracy] ==> 1.0
[Valid loss] ==> 0.7129944562911987    [Valid Accuracy] ==> 0.800000011920929
[3/50]
TRAIN : 0.7544456124305725
VAL : 0.7072577476501465
[Train loss] ==> 0.6554285287857056    [Train Accuracy] ==> 1.0
[Valid loss] ==> 0.6253121495246887    [Valid Accuracy] ==> 1.0
[4/50]
TRAIN : 0.664445698261261
VAL : 0.6178791522979736
[Train loss] ==> 0.5685132145881653    [Train Accuracy] ==> 1.0
[Valid loss] ==> 0.5727471709251404    [Valid Accuracy] ==> 1.0
[5/50]
TRAIN : 0.5814414620399475
VAL : 0.5563734769821167
[Train loss] ==> 0.499